In [1]:
# Modelių ir prekybos strategijų hiperparametrų optimizavimas (BTC/USDT)
#
# Šiame notebook bus testuojami įvairūs neuroninių tinklų hiperparametrai (epochs, batch_size, learning_rate, dropout ir kt.)
# Taip pat bus testuojami prekybos strategijų parametrai (stop-loss, take-profit, pozicijos dydis).
# Visi rezultatai bus įrašyti į ataskaitos lentelę.

In [2]:
# Reikalingų bibliotekų importavimas
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import random
import os

np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Duomenų paruošimas (naudojamas tas pats kaip kituose modeliuose)
import requests
from datetime import datetime, timedelta

def get_historical_klines(symbol="BTCUSDT", interval="15m", start_time=None, end_time=None):
    BINANCE_API_URL = "https://api.binance.com/api/v3/klines"
    if end_time is None:
        end_time = datetime.now()
    if start_time is None:
        start_time = end_time - timedelta(days=365)
    start_ts = int(start_time.timestamp() * 1000)
    end_ts = int(end_time.timestamp() * 1000)
    all_klines = []
    current_start = start_ts
    while current_start < end_ts:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': current_start,
            'endTime': end_ts,
            'limit': 1000
        }
        try:
            response = requests.get(BINANCE_API_URL, params=params)
            response.raise_for_status()
            klines = response.json()
            if not klines:
                break
            all_klines.extend(klines)
            current_start = int(klines[-1][0]) + 1
        except Exception as e:
            print(f"Klaida gaunant duomenis: {str(e)}")
    if all_klines:
        columns = ['time', 'open', 'high', 'low', 'close', 'volume', 
                   'close_time', 'quote_asset_volume', 'number_of_trades',
                   'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
        df = pd.DataFrame(all_klines, columns=columns)
        df['time'] = pd.to_datetime(df['time'], unit='ms')
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        df[numeric_columns] = df[numeric_columns].astype(float)
        return df
    return None

df = get_historical_klines()
df = df.sort_values('time')
columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

def create_sequences(data, target_column, sequence_length):
    X, y = [], []
    feature_columns = columns_to_normalize
    data_array = data[feature_columns].values
    target_idx = feature_columns.index(target_column)
    for i in range(len(data) - sequence_length):
        X.append(data_array[i:i + sequence_length])
        y.append(data_array[i + sequence_length, target_idx])
    return np.array(X), np.array(y)

sequence_length = 10
target_column = 'close'
X, y = create_sequences(df, target_column, sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [4]:
# Hiperparametrų tinklelis (galite keisti pagal poreikį)
param_grid = {
    'epochs': [10, 20, 30],
    'batch_size': [16, 32, 64],
    'learning_rate': [0.001, 0.0005],
    'dropout': [0.1, 0.2, 0.3],
    'units': [32, 64],
    'optimizer': ['adam', 'rmsprop']
}
grid = list(ParameterGrid(param_grid))
random.shuffle(grid)
grid = grid[:30]  # Pasirenkame 30 variantų
print(f"Bus testuojama {len(grid)} hiperparametrų kombinacijų.")

Bus testuojama 30 hiperparametrų kombinacijų.


In [5]:
# Paprastas LSTM modelio kūrimo šablonas
def build_lstm_model(input_shape, units=64, dropout=0.2, learning_rate=0.001, optimizer='adam'):
    model = models.Sequential()
    model.add(layers.LSTM(units, input_shape=input_shape, return_sequences=False))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1))
    if optimizer == 'adam':
        opt = optimizers.Adam(learning_rate=learning_rate)
    else:
        opt = optimizers.RMSprop(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mse')
    return model

In [6]:
# Prekybos strategijos simuliatorius
def trading_simulator(y_true, y_pred, stop_loss=0.03, take_profit=0.05, position_size=1.0, fee=0.001):
    balance = 1000.0
    btc = 0.0
    last_action = None
    for i in range(1, len(y_pred)):
        price_now = y_true[i-1]
        price_next = y_true[i]
        # Signalas: jei prognozė > dabartinė kaina -> pirkti, jei < -> parduoti
        if y_pred[i] > price_now and last_action != 'buy':
            btc = balance * position_size * (1 - fee) / price_now
            balance -= balance * position_size
            last_action = 'buy'
            entry_price = price_now
        elif y_pred[i] < price_now and last_action == 'buy':
            balance += btc * (1 - fee) * price_now
            btc = 0
            last_action = 'sell'
        # Stop-loss / take-profit
        if last_action == 'buy':
            if (price_now - entry_price) / entry_price <= -stop_loss or (price_now - entry_price) / entry_price >= take_profit:
                balance += btc * (1 - fee) * price_now
                btc = 0
                last_action = 'sell'
    if btc > 0:
        balance += btc * (1 - fee) * y_true[-1]
    return balance

In [7]:
# Testuojame 30+ hiperparametrų kombinacijų ir prekybos strategijų
results = []
for i, params in enumerate(grid):
    print(f"Bandymas {i+1}/{len(grid)}: {params}")
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        units=params['units'],
        dropout=params['dropout'],
        learning_rate=params['learning_rate'],
        optimizer=params['optimizer']
    )
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_data=(X_test, y_test),
        verbose=0
    )
    y_pred = model.predict(X_test)
    # Atstatome į originalią skalę
    dummy = np.zeros((len(y_pred), len(columns_to_normalize)))
    dummy[:, columns_to_normalize.index(target_column)] = y_pred.flatten()
    y_pred_original = scaler.inverse_transform(dummy)[:, columns_to_normalize.index(target_column)]
    dummy_y = np.zeros((len(y_test), len(columns_to_normalize)))
    dummy_y[:, columns_to_normalize.index(target_column)] = y_test.flatten()
    y_test_original = scaler.inverse_transform(dummy_y)[:, columns_to_normalize.index(target_column)]
    # ML metrikos
    mse = mean_squared_error(y_test_original, y_pred_original)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_original, y_pred_original)
    r2 = r2_score(y_test_original, y_pred_original)
    mape = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100
    # Prekybos strategijos parametrai (galite randomizuoti arba fiksuoti)
    stop_loss = random.choice([0.01, 0.02, 0.03, 0.05])
    take_profit = random.choice([0.03, 0.05, 0.07, 0.1])
    position_size = random.choice([0.5, 1.0])
    final_balance = trading_simulator(y_test_original, y_pred_original, stop_loss, take_profit, position_size)
    results.append({
        **params,
        'stop_loss': stop_loss,
        'take_profit': take_profit,
        'position_size': position_size,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'final_balance': final_balance
    })

Bandymas 1/30: {'batch_size': 64, 'dropout': 0.3, 'epochs': 30, 'learning_rate': 0.001, 'optimizer': 'adam', 'units': 64}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Bandymas 2/30: {'batch_size': 16, 'dropout': 0.2, 'epochs': 10, 'learning_rate': 0.0005, 'optimizer': 'rmsprop', 'units': 64}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Bandymas 3/30: {'batch_size': 16, 'dropout': 0.1, 'epochs': 10, 'learning_rate': 0.0005, 'optimizer': 'rmsprop', 'units': 32}


...

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Bandymas 29/30: {'batch_size': 64, 'dropout': 0.2, 'epochs': 30, 'learning_rate': 0.001, 'optimizer': 'adam', 'units': 32}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Bandymas 30/30: {'batch_size': 32, 'dropout': 0.1, 'epochs': 10, 'learning_rate': 0.0005, 'optimizer': 'rmsprop', 'units': 32}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [8]:
# Rezultatų ataskaitos lentelė
df_results = pd.DataFrame(results)
display(df_results.sort_values('final_balance', ascending=False).head(10))
df_results.to_csv('../../models/hyperparameter_optimization_results.csv', index=False)
print("Rezultatai išsaugoti į ../../models/hyperparameter_optimization_results.csv")

,batch_size,dropout,epochs,learning_rate,optimizer,units,stop_loss,take_profit,position_size,rmse,mae,r2,mape,final_balance
27,16,0.1,20,0.0005,adam,64,0.03,0.07,0.5,467.793855,407.433835,0.996758,0.449874,1018.777820
4,16,0.2,20,0.0010,rmsprop,32,0.02,0.10,1.0,1025.822974,936.982588,0.984411,1.019567,1002.079551
8,32,0.2,10,0.0010,rmsprop,32,0.02,0.05,0.5,1016.464597,917.675041,0.984694,0.997079,991.806697
6,16,0.2,20,0.0010,rmsprop,64,0.05,0.07,1.0,673.970334,609.908593,0.993271,0.668011,971.683682
1,16,0.2,10,0.0005,rmsprop,64,0.01,0.07,0.5,761.411167,709.019898,0.991411,0.788784,967.566058
10,64,0.3,10,0.0010,rmsprop,64,0.02,0.05,1.0,920.564527,842.408975,0.987446,0.920368,966.689299
2,16,0.1,10,0.0005,rmsprop,32,0.01,0.03,0.5,837.213100,772.904694,0.989616,0.849926,945.101560
26,16,0.3,20,0.0010,rmsprop,64,0.03,0.10,0.5,399.171204,323.623430,0.997640,0.358626,832.756136
16,32,0.2,10,0.0010,adam,32,0.01,0.03,1.0,802.891741,699.871083,0.990450,0.758378,808.959006
14,32,0.2,10,0.0005,adam,64,0.01,0.07,0.5,427.489672,346.463046,0.997293,0.385243,792.119111


Rezultatai išsaugoti į ../../models/hyperparameter_optimization_results.csv
